In [5]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain_ollama.llms import OllamaLLM

load_dotenv()
model = OllamaLLM(model="llama3.2:latest")
print("✅模型已建立")

✅模型已建立


In [6]:
# 主提示模板
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "您是專業的客服代表。"),
        ("human", "請回覆這封客戶郵件:\n\n{email_content}")
    ]
)

print("✅ 主提示模板已建立")



✅ 主提示模板已建立


In [7]:
def prepare_quality_check_prompt(reply):
    """
    根據回覆內容動態準備品質檢查 Prompt
    
    關鍵點：
    - 只負責準備數據，不直接調用模型
    - 返回 dict 格式，傳給下一個 Prompt Template
    - 根據內容動態調整檢查重點
    """

    #步驟1:格式化郵件
    formatted = f"""
親愛的客戶,
{reply}

此致
客服團隊
"""
    # 步驟 2:智能分析內容,決定檢查重點
    if "退貨" in reply or "換貨" in reply:
        focus = "請特別注意退換貨政策的說明是否清楚、完整。評估是否明確告知客戶處理流程。"
    elif "抱歉" in reply or "歉意" in reply:
        focus = "請特別注意道歉的誠意和補償措施。評估是否展現足夠的同理心和解決問題的決心。"
    elif "感謝" in reply:
        focus = "請評估回覆的友善度和專業度。確認是否適當表達了對客戶的重視。"
    else:
        focus = "請評估整體的專業度、友善度和完整度。"

    return {
        "email":formatted,
        "focus":focus
    }

In [8]:
# 品質檢查Prompt(接收動態準備的數據)
dynamic_quality_propmt = ChatPromptTemplate.from_messages(
    [
        ("system","你是郵件品質審查專家。{focus}"),
        ("human", "評估這封郵件:\n\n{email}")
    ]
)
dynamic_chain = (
    prompt_template
    | model
    | StrOutputParser()
    | RunnableLambda(prepare_quality_check_prompt)
    | dynamic_quality_propmt
    | model
    | StrOutputParser()
)
print("✅ 動態提示鏈已建立完成！")
print("\n流程：主 Prompt → 主模型 → 動態準備檢查 Prompt → 品質檢查模型")
print("\n✨ 特點：Lambda 只負責準備數據，模型調用清晰可見")

✅ 動態提示鏈已建立完成！

流程：主 Prompt → 主模型 → 動態準備檢查 Prompt → 品質檢查模型

✨ 特點：Lambda 只負責準備數據，模型調用清晰可見


In [9]:
# 測試案例 1：退換貨問題
customer_email_1 = """
您好，

我最近購買了貴公司的產品，但是發現包裝有損壞。
請問可以退貨或換貨嗎？

王小明
"""

print("="*60)
print("測試案例 1：退換貨問題（應強調退換貨政策）")
print("="*60)
result1 = dynamic_chain.invoke({"email_content": customer_email_1})
print(result1)

測試案例 1：退換貨問題（應強調退換貨政策）
評估結果：

退換貨政策的說明:

* 回應清晰且提到退換貨政策，告知客戶可以優先處理退貨或換貨要求。
* 提供了包裝受損的情況下會提供全新的產品和退貨oucher的解釋。

處理流程未被明確告知:

* 未提到退貨或換貨的程序詳細，例如要填寫什麼單位、需要提供什麼證據等。
* 未明確告知客戶該如何聯繫客服代表進行進一步協助。

評分：6/10

建議:

* 將退換貨政策的說明作為一個獨立的部分，例如 "退換貨政策"。
* 提供清晰且詳細的處理流程，包括填寫什麼單位、需要提供什麼證據等。
* 加強了客戶與客服代表之間的溝通機會，例如提供聯繫方式和Contact 時間。


In [10]:
# 測試案例 2：投訴問題
customer_email_2 = """
你們的服務態度太差了！
我要投訴！
"""

print("\n" + "="*60)
print("測試案例 2：投訴問題（應強調道歉誠意）")
print("="*60)
result2 = dynamic_chain.invoke({"email_content": customer_email_2})
print(result2)


測試案例 2：投訴問題（應強調道歉誠意）
對於這封郵件，我給予友善度和專業度的評分如下：

友善度：8/10

* 這封郵件對客戶表達了 sincerelly 的歉意和關心，這是很好的開始。
* 使用親愛的語氣和尊敬的語氣，表明對客戶的重視。
* 提供電話/Email 狀況，並提供幫助資訊，有助於客戶快速找到解決方案。

專業度：9/10

* 回覆中使用了正確的語句和 格式，表明了熟練的寫作技能。
* 應該將 "尊敬的customer" 和 "親愛的客戶" 的字樣用一致的語氣來表達，以避免混淆。
* 回覆中缺少了一些詳細的資訊，例如問題描述或具體的解決方案，可能需要更多的信息以完成處理。

適當表達對客戶的重視：9/10

* 這封郵件表明了對客戶的關心和歉意，並提供幫助資訊，有助於客戶快速找到解決方案。
* 使用了 "我們值得您的信任和支持" 的語句，強調了公司的重視對客戶。

總體來說，這封郵件是很好的回覆，但還需要一些微調以提高其專業度和具體性。


In [11]:
# 測試案例 3：一般諮詢
customer_email_3 = """
請問你們的營業時間？
"""

print("\n" + "="*60)
print("測試案例 3：一般諮詢（評估整體專業度）")
print("="*60)
result3 = dynamic_chain.invoke({"email_content": customer_email_3})
print(result3)


測試案例 3：一般諮詢（評估整體專業度）
根據我的評估，這封郵件的專業度為 8/10。原因如下：

- 專業度：郵件中使用了簡潔且明確的語句，表達了營業時間、休息日等必要資訊。
-友善度：雖然郵件是正式的，但 tone 感覺相當友善和幫助性，可以讓客戶了解公司的OPERATING 時間和查找服務資訊的途徑。

然而，整體而言，這封郵件缺乏了一點完整度。例如，雖然提到了對客戶需要任何服務或問卷時應該如何尋找相關資訊，但没有提供明確的連結或接觸方式來進行查找，這使得這個部分的完整度感覺不夠完善。

綜合評估後，我給這封郵件的整體評分為 7.5/10。
